# Collect Neurons

This notebook downloads corresponding neurons from two versions of microns and compares their skeletal length to discover neurons that may have had merge errors in earlier releases

## Imports and functions

In [1]:
from mesh_tools import trimesh_utils as tu
from datasci_tools import ipyvolume_utils as ipvu
from neurd import neuron_visualizations as nviz
from pathlib import Path
from cloudvolume import CloudVolume
import trimesh
from neurd.vdi_microns import volume_data_interface as vdi

INFO - 2024-08-28 11:43:42,358 - utils - NumExpr defaulting to 8 threads.
/usr/local/lib/python3.8/dist-packages/python_jsonschema_objects/__init__.py:113: UserWarning: Schema id not specified. Defaulting to 'self'
  warnings.warn("Schema id not specified. Defaulting to 'self'")


In [2]:
import ipyvolume as ipv
import ipyvolume.pylab as p3
import shutil
import numpy as np
from tqdm import tqdm
import pandas as pd

## Old version: only valid for neurons where ID does not change in manifestations

In [3]:
def collect_meshes(id, cv_new, cv_old, decimation_parameters = dict(decimation_ratio = 0.25*0.25)):
    # get newer version of neuron
    mesh_new = cv_new.mesh.get(id)[id]
    # decimate for runtime
    mesh_new_decimated = tu.decimate(
        mesh_new,
        **decimation_parameters
    )
    # same for old neuron
    mesh_old = cv_old.mesh.get(id)[id]
    mesh_old_decimated = tu.decimate(
        mesh_old,
        **decimation_parameters
    )

    return mesh_new_decimated, mesh_old_decimated

In [4]:
# oldest manifestation
cv_old = CloudVolume('precomputed://https://storage.googleapis.com/iarpa_microns/minnie/minnie65/seg',
                    progress=False, # shows progress bar
                    cache=False, # cache to disk to avoid repeated downloads
                    fill_missing=False,)
# most recent manifestation
cv_new = CloudVolume('graphene://https://minnie.microns-daf.com/segmentation/table/minnie65_public',
                    progress=False, # shows progress bar
                    cache=False, # cache to disk to avoid repeated downloads
                    fill_missing=False,)

Using default Google credentials. There is no ~/.cloudvolume/secrets/google-secret.json set.

        google-cloud-python might not find your credentials. Your credentials
        might be located in $HOME/.config/gcloud/legacy_credentials/$YOUR_GMAIL/adc.json

        If they are you can export your credentials like so:
        export GOOGLE_APPLICATION_CREDENTIALS="$HOME/.config/gcloud/legacy_credentials/$YOUR_GMAIL/adc.json"
      


/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
WARNING - 2024-08-28 11:43:53,460 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable


In [4]:
# ids that remain the same through both releases
good_ids = [864691135975539779, 864691135975633475, 864691135497743635, 864691136108938168, 864691135544588584, 864691136023889209, 864691135212863360,
 864691136309871706, 864691136903144370, 864691135915343462, 864691135700409211, 864691135593659947, 864691135699487522, 864691136023980601, 864691135855890478,
 864691135952122147, 864691136209328060, 864691136812081779, 864691136136778621, 864691135396580129, 864691136237644604, 864691135359010904, 864691136105661657,
 864691135864188286, 864691135407429321, 864691136334080435, 864691135654280386, 864691136136766845, 864691135918528432, 864691135012753654, 864691136311902269,
 864691136031910331, 864691135367232761, 864691135081877495, 864691137019755118, 864691135358995288, 864691135753932237, 864691135848030814, 864691134884807418,
 864691135440543560, 864691135233242713, 864691136925601354, 864691135517531786, 864691135782544435, 864691135724393131, 864691135272206865, 864691135373601736,
 864691135953985800, 864691136123919654, 864691136100014453, 864691136118618136, 864691135609687047, 864691135462420637, 864691136618564493, 864691135382556378,
 864691135360096071, 864691135577202181, 864691136451224831, 864691135809608652, 864691135099901728, 864691135888577417, 864691135463695429, 864691135501703746,
 864691135385422677, 864691135341198789, 864691135367033849, 864691135117980637, 864691135508871945, 864691136378815445, 864691135937286404, 864691135428676144,
 864691137019596142, 864691135952121379, 864691136194355176, 864691136136766589, 864691136143696692, 864691135754881746, 864691135739733140, 864691135065145924,
 864691135775906989, 864691136084313196, 864691136227020113, 864691135113360025, 864691136099960181, 864691135692949951, 864691135855860782, 864691135367305465,
 864691136118618392, 864691135122469671]

In [ ]:
results = np.zeros((len(good_ids), 4))

for i, id in tqdm(enumerate(good_ids)):
    mesh_new, mesh_old = collect_meshes(id, cv_new, cv_old)
    length_new = tu.skeletal_length_from_mesh(trimesh.Trimesh(mesh_new.vertices, mesh_new.faces))
    length_old = tu.skeletal_length_from_mesh(trimesh.Trimesh(mesh_old.vertices, mesh_old.faces))

    results[i, 0] = length_new
    results[i, 1] = length_old
    # difference in lengths
    results[i, 2] = length_new - length_old
    # scale
    results[i, 3] = length_new/length_old

    # manually remove temp folder to save disk space
    shutil.rmtree('temp/')

np.save("mesh_length_comparisons_new-old-diff-ratio", results)

0it [00:00, ?it/s]
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
WARNING - 2024-08-23 15:05:07,512 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
WARNING - 2024-08-23 15:05:07,961 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials

xvfb-run -n 7939 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_49663.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_49663_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06398610.mls



Upload to Cache: 0it [00:00, ?it/s]


xvfb-run -n 116 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_47675.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_47675_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06249228.mls


1it [14:27, 867.96s/it]

...

xvfb-run -n 4057 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_78183.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_78183_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06835084.mls
xvfb-run -n 4381 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_84629.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_84629_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06948286.mls


7it [1:34:14, 851.52s/it]

xvfb-run -n 2086 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_22244.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_22244_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06798928.mls
xvfb-run -n 889 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_28961.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_28961_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06478157.mls


8it [1:46:32, 815.37s/it]

xvfb-run -n 7500 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_55416.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_55416_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06331763.mls
xvfb-run -n 3494 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_1559.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_1559_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_0665489.mls


In [ ]:
# Failed first time around, so continue where it left off
results = np.load("mesh_length_comparisons_new-old-diff-ratio.npy")

for i in range(67, 89):
    id = good_ids[i]
    
    mesh_new, mesh_old = collect_meshes(id, cv_new, cv_old)
    length_new = tu.skeletal_length_from_mesh(trimesh.Trimesh(mesh_new.vertices, mesh_new.faces))
    length_old = tu.skeletal_length_from_mesh(trimesh.Trimesh(mesh_old.vertices, mesh_old.faces))

    results[i, 0] = length_new
    results[i, 1] = length_old
    results[i, 2] = length_new - length_old
    results[i, 3] = length_new/length_old

    # manually remove temp folder
    shutil.rmtree('temp/')

np.save("mesh_length_comparisons_new-old-diff-ratio", results)

  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
WARNING - 2024-08-26 09:57:32,464 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
WARNING - 2024-08-26 09:57:32,924 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud 

xvfb-run -n 5263 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_64165.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_64165_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06171060.mls
xvfb-run -n 3296 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_93533.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_93533_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06593459.mls


xvfb-run -n 3243 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_75907.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_75907_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06264931.mls
xvfb-run -n 8086 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_54641.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_54641_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06362250.mls


xvfb-run -n 3903 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_95749.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_95749_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06309634.mls
xvfb-run -n 1084 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_71038.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_71038_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06834743.mls


xvfb-run -n 9297 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_28703.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_28703_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06117820.mls
xvfb-run -n 9012 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_65035.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_65035_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06989411.mls


...

xvfb-run -n 1792 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_75326.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_75326_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06697434.mls
xvfb-run -n 8026 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_36810.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_36810_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06155309.mls


xvfb-run -n 7502 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_33414.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_33414_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_0668547.mls
xvfb-run -n 9449 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_99942.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_99942_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06410880.mls


xvfb-run -n 9651 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_62038.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_62038_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06578516.mls
xvfb-run -n 3962 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_17435.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_17435_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06290582.mls


xvfb-run -n 1502 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_91056.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_91056_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06163189.mls


xvfb-run -n 7704 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_77649.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_77649_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06432290.mls


In [26]:
results = np.load("mesh_length_comparisons_new-old-diff-ratio.npy")

## Better: for Microns neurons with many changes

root_ids_proofread_microns.csv is produced by collect_cave_changes.ipynb

In [6]:
root_ids = pd.read_csv("root_ids_proofread_microns.csv")

In [10]:
root_ids

,Unnamed: 0,root_id_1078,pt_supervoxel_id,pt_position_x,pt_position_y,pt_position_z,n_changes,root_id_117
0,51,864691135335733481,90924527114284243,190027,121508,20685,2377,864691135660642800
1,54,864691135952122147,112175063088115757,344768,115350,19786,1714,864691135952122147
2,2,864691135082864887,106552435166206532,303659,166262,17349,1183,864691135609594119
3,6,864691136144674612,88815663879226337,174921,137614,21123,1114,864691135293126156
4,14,864691135488888378,90571584016011523,187568,113216,21732,979,864691135925753358
5,19,864691136023889209,87904100087286054,168181,161960,21511,935,864691136023889209
6,32,864691136990522517,93177907207497424,206577,133261,18640,838,864691136008689326
7,36,864691136137834877,106053051533554699,300576,115616,23080,788,864691135415666362
8,1316,864691135123617831,95641569704922650,224336,138864,22805,752,864691135502241717
9,48,864691136023980601,105004461172496411,292612,167390,24276,732,864691136023980601


In [8]:
decimation_parameters = dict(decimation_ratio = 0.25*0.25)

n_neurons = root_ids.shape[0]

In [ ]:
results_largechanges = pd.DataFrame(columns=["pt_supervoxel_id", "root_id_1078", "root_id_117", "length_1078", "length_117", "length_difference", "length_ratio_new_old"])

# for each neuron
for i in tqdm(range(n_neurons)):
    # all known info from previous dataframe
    neuron_info = root_ids.iloc[i,:]
    
    results_largechanges.loc[i,"pt_supervoxel_id"] = neuron_info.pt_supervoxel_id
    # root id in most recent manifestation (v1078)
    results_largechanges.loc[i,"root_id_1078"] = neuron_info.root_id_1078
    # root id in oldest manifestation (v117)
    results_largechanges.loc[i,"root_id_117"] = neuron_info.root_id_117

    # download mesh and decimate for runtime
    mesh_new = cv_new.mesh.get(neuron_info.root_id_1078)[neuron_info.root_id_1078]
    mesh_new = tu.decimate(
        mesh_new,
        **decimation_parameters
    )

    mesh_old = cv_old.mesh.get(neuron_info.root_id_117)[neuron_info.root_id_117]
    mesh_old = tu.decimate(
        mesh_old,
        **decimation_parameters
    )

    # get length
    length_new = tu.skeletal_length_from_mesh(trimesh.Trimesh(mesh_new.vertices, mesh_new.faces))
    length_old = tu.skeletal_length_from_mesh(trimesh.Trimesh(mesh_old.vertices, mesh_old.faces))

    # save length statistics
    results_largechanges.loc[i, "length_1078"] = length_new
    results_largechanges.loc[i, "length_117"] = length_old
    results_largechanges.loc[i, "length_difference"] = length_new - length_old
    results_largechanges.loc[i, "length_ratio_new_old"] = length_new/length_old

    # free up disk space
    shutil.rmtree('temp/')

results_largechanges.to_csv('large_changes_length_comparison.csv')

In [7]:
results_largechanges

,pt_supervoxel_id,root_id_1078,root_id_117,length_1078,length_117,length_difference,length_ratio_new_old
0,90924527114284243,864691135335733481,864691135660642800,13849712.128857,1185021.762194,12664690.366663,11.687306
1,112175063088115757,864691135952122147,864691135952122147,9754890.39446,9179331.548589,575558.845871,1.062702
2,106552435166206532,864691135082864887,864691135609594119,9723653.055064,8775256.048868,948397.006196,1.108076
3,88815663879226337,864691136144674612,864691135293126156,6849780.032073,6518367.683717,331412.348356,1.050843
4,90571584016011523,864691135488888378,864691135925753358,5614532.720058,542114.263663,5072418.456395,10.356733
5,87904100087286054,864691136023889209,864691136023889209,8191144.417017,8206055.971406,-14911.55439,0.998183
6,93177907207497424,864691136990522517,864691136008689326,8475854.835295,7805098.562045,670756.273251,1.085938
7,106053051533554699,864691136137834877,864691135415666362,8032883.287099,7066126.465602,966756.821497,1.136816
8,95641569704922650,864691135123617831,864691135502241717,7060461.597809,999506.22208,6060955.375729,7.06395
9,105004461172496411,864691136023980601,864691136023980601,6175149.676295,5586886.628677,588263.047618,1.105294


In [9]:
# Collect meshes from both manifestations for visualisation
for i in tqdm(range(n_neurons-5)):
    neuron_info = root_ids.iloc[i+5,:]

    mesh_new = cv_new.mesh.get(neuron_info.root_id_1078)[neuron_info.root_id_1078]
    mesh_new = tu.decimate(
        mesh_new,
        **decimation_parameters
    )
    with open(f"microns_new/{neuron_info.root_id_1078}.obj", "wb") as f:
        trimesh.exchange.export.export_mesh(mesh_new, f, file_type='obj')
    
    mesh_old = cv_old.mesh.get(neuron_info.root_id_117)[neuron_info.root_id_117]
    mesh_old = tu.decimate(
        mesh_old,
        **decimation_parameters
    )
    with open(f"microns_old/{neuron_info.root_id_117}.obj", "wb") as f:
        trimesh.exchange.export.export_mesh(mesh_old, f, file_type='obj')

    shutil.rmtree('temp/')

  0%|                                                                                      | 0/15 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)
WARNING - 2024-08-28 11:44:26,737 - _default - No project ID could be determined. Consider running `gcloud config set project` or setting the GOOGLE_CLOUD_PROJECT environment variable
/usr/local/lib/python3.8/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page f

xvfb-run -n 2599 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_21565.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_21565_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06235582.mls
xvfb-run -n 3251 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_84656.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_84656_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06340822.mls


  7%|█████                                                                      | 1/15 [06:36<1:32:24, 396.03s/it]

xvfb-run -n 8149 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_82987.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_82987_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06478423.mls
xvfb-run -n 4129 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_6930.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_6930_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_0657479.mls


 13%|██████████                                                                 | 2/15 [17:26<1:58:14, 545.72s/it]

...

 87%|████████████████████████████████████████████████████████████████▏         | 13/15 [2:02:48<16:22, 491.43s/it]

xvfb-run -n 5555 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_69354.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_69354_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_0691038.mls
xvfb-run -n 8976 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_22161.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_22161_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06765008.mls


 93%|█████████████████████████████████████████████████████████████████████     | 14/15 [2:06:42<06:53, 413.86s/it]

xvfb-run -n 780 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_80042.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_80042_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06243105.mls
xvfb-run -n 6550 -s "-screen 0 800x600x24" meshlabserver $@  -i /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_43319.off -o /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/neuron_43319_decimated.off -s /NEURD/Applications/Tutorials/Auto_Proof_Pipeline/Single_Soma_Exc/temp/decimation_meshlab_06258784.mls


100%|██████████████████████████████████████████████████████████████████████████| 15/15 [2:15:55<00:00, 543.70s/it]
